!pip3 uninstall -y torch torchvision

!pip3 install torch torchvision

In [1]:
import torch
# import the data
import pandas as pd

In [3]:
import os
os.environ["KERAS_BACKEND"] = "torch"

In [5]:
import numpy as np
import keras
from keras import layers
# from tensorflow import data as tf_data
# import matplotlib.pyplot as plt

In [7]:
# split it like above

# do the shaping if necessary
# change the type of the columns that are characters to one-hot-encodings or something like that

# make a model with one layer
# run the model and see what happens

In [9]:
df = pd.read_csv('data\\online_shoppers_intention.csv')

print(df)

# month, visitor_type, weekend, and revenue need to be converted
# needs to be a numpy array befor being provided as input, so need to make everything encoded (we can try both one-hot and numerical)

#from numpy import genfromtxt
#my_data = genfromtxt('data\\online_shoppers_intention.csv', delimiter=',')

#print(my_data)

       Administrative  Administrative_Duration  Informational  \
0                   0                      0.0              0   
1                   0                      0.0              0   
2                   0                      0.0              0   
3                   0                      0.0              0   
4                   0                      0.0              0   
...               ...                      ...            ...   
12325               3                    145.0              0   
12326               0                      0.0              0   
12327               0                      0.0              0   
12328               4                     75.0              0   
12329               0                      0.0              0   

       Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                         0.0               1                 0.000000   
1                         0.0               2                64.000000 

In [11]:
df['total_duration'] = df['Administrative_Duration'] + df['Informational_Duration'] + df['ProductRelated_Duration']
df['proportion_of_duration'] = df['ProductRelated_Duration'] / df['total_duration']

In [13]:
df.drop(columns=['Informational_Duration', 'Administrative_Duration', 
                 'ProductRelated_Duration', 'Administrative', 'Informational', 
                 'ProductRelated', 'OperatingSystems', 'Browser'])

,BounceRates,ExitRates,PageValues,SpecialDay,Month,Region,TrafficType,VisitorType,Weekend,Revenue,total_duration,proportion_of_duration
0,0.200000,0.200000,0.000000,0.0,Feb,1,1,Returning_Visitor,False,False,0.000000,NaN
1,0.000000,0.100000,0.000000,0.0,Feb,1,2,Returning_Visitor,False,False,64.000000,1.000000
2,0.200000,0.200000,0.000000,0.0,Feb,9,3,Returning_Visitor,False,False,0.000000,NaN
3,0.050000,0.140000,0.000000,0.0,Feb,2,4,Returning_Visitor,False,False,2.666667,1.000000
4,0.020000,0.050000,0.000000,0.0,Feb,1,4,Returning_Visitor,True,False,627.500000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
12325,0.007143,0.029031,12.241717,0.0,Dec,1,1,Returning_Visitor,True,False,1928.791667,0.924823
12326,0.000000,0.021333,0.000000,0.0,Nov,1,8,Returning_Visitor,True,False,465.750000,1.000000
12327,0.083333,0.086667,0.000000,0.0,Nov,1,13,Returning_Visitor,True,False,184.250000,1.000000
12328,0.000000,0.021053,0.000000,0.0,Nov,3,11,Returning_Visitor,False,False,421.000000,0.821853


In [15]:
# df['Month'].replace(['
print(df['Month'].unique())

['Feb' 'Mar' 'May' 'Oct' 'June' 'Jul' 'Aug' 'Nov' 'Sep' 'Dec']


In [17]:
df['Month'].replace(['Jan','Feb','Mar','Apr','May','June','Jul','Aug','Sep','Oct','Nov','Dec'],
                    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], inplace=True)

C:\Users\tomli\AppData\Local\Temp\ipykernel_30816\727251348.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Month'].replace(['Jan','Feb','Mar','Apr','May','June','Jul','Aug','Sep','Oct','Nov','Dec'],
C:\Users\tomli\AppData\Local\Temp\ipykernel_30816\727251348.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_dow

In [19]:
print(df['Month'].unique())

[ 2  3  5 10  6  7  8 11  9 12]


In [21]:
print(df['VisitorType'].unique())

['Returning_Visitor' 'New_Visitor' 'Other']


In [23]:
df['VisitorType'].replace(['Returning_Visitor', 'New_Visitor', 'Other'], [0, 1, 2], inplace=True)

C:\Users\tomli\AppData\Local\Temp\ipykernel_30816\656729720.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['VisitorType'].replace(['Returning_Visitor', 'New_Visitor', 'Other'], [0, 1, 2], inplace=True)
C:\Users\tomli\AppData\Local\Temp\ipykernel_30816\656729720.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_d

In [25]:
df_copy = df.copy(deep=True)

In [27]:
df_copy['Weekend'].replace([False, True], [0,1], inplace=True)

C:\Users\tomli\AppData\Local\Temp\ipykernel_30816\2402203070.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_copy['Weekend'].replace([False, True], [0,1], inplace=True)
C:\Users\tomli\AppData\Local\Temp\ipykernel_30816\2402203070.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_copy['We

In [29]:
df_copy['Revenue'].replace([False, True], [0,1], inplace=True)

C:\Users\tomli\AppData\Local\Temp\ipykernel_30816\550847130.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_copy['Revenue'].replace([False, True], [0,1], inplace=True)
C:\Users\tomli\AppData\Local\Temp\ipykernel_30816\550847130.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_copy['Reve

In [31]:
df_copy = df_copy.drop(columns=['Informational_Duration', 'Administrative_Duration', 
                 'ProductRelated_Duration', 'Administrative', 'Informational', 
                 'ProductRelated', 'OperatingSystems', 'Browser'])

In [33]:
df_copy.fillna(1, inplace=True)

In [35]:
df_numpy = df_copy.to_numpy()

In [37]:
import math
df_numpy.shape[0]
testSample = np.random.choice(df_numpy.shape[0], math.floor(0.7 * df_numpy.shape[0]), replace=False)

In [39]:
print(testSample)

[ 7797  8357  6218 ...  8113  1616 12184]


In [41]:
print(df_numpy[testSample])

train = df_numpy[testSample]
test = df_numpy[~testSample]

print("Train length: " + str(train.shape[0]))
print("Test lenght: " + str(test.shape[0]))

[[0.00000000e+00 1.92307690e-02 0.00000000e+00 ... 0.00000000e+00
  5.64946667e+02 6.91138987e-01]
 [1.11111110e-02 5.00000000e-02 1.79606904e+01 ... 1.00000000e+00
  3.54750000e+02 8.97110641e-01]
 [0.00000000e+00 1.17647060e-02 0.00000000e+00 ... 0.00000000e+00
  4.18100000e+02 6.79502511e-01]
 ...
 [0.00000000e+00 6.89655200e-03 0.00000000e+00 ... 1.00000000e+00
  4.62500000e+02 9.39459459e-01]
 [0.00000000e+00 7.14285700e-03 0.00000000e+00 ... 0.00000000e+00
  4.06866667e+02 1.00000000e+00]
 [0.00000000e+00 6.91029900e-03 3.33122059e+01 ... 0.00000000e+00
  2.33980000e+03 4.32615323e-01]]
Train length: 8631
Test lenght: 8631


In [43]:
from sklearn.model_selection import train_test_split

In [44]:
df_copy_y = df_copy['Revenue']
df_copy_x = df_copy.loc[:, df_copy.columns != 'Revenue']

#print(df_copy_y)
#print(df_copy_x)

In [47]:
df_copy_x_train, df_copy_x_test, df_copy_y_train, df_copy_y_test = train_test_split(
    df_copy_x, df_copy_y, test_size=0.3, random_state=42)

In [49]:
# print("df_copy_x_train shape:", df_copy_x_train.shape)
# print(df_copy_x_train.to_numpy())

numpy_x_train = df_copy_x_train.to_numpy()
numpy_x_test = df_copy_x_test.to_numpy()
numpy_y_train = df_copy_y_train.to_numpy()
numpy_y_test = df_copy_y_test.to_numpy()

num_classes = 2

input_shape = (11, 1)

# Make sure images have shape (28, 28, 1)
numpy_x_train = np.expand_dims(numpy_x_train, -1)
numpy_x_test = np.expand_dims(numpy_x_test, -1)
print("x_train shape:", numpy_x_train.shape)
print(numpy_x_train.shape[0], "train samples")
print(numpy_x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
numpy_y_train = keras.utils.to_categorical(numpy_y_train, num_classes)
numpy_y_test = keras.utils.to_categorical(numpy_y_test, num_classes)

print("y_train shape: ", numpy_y_train.shape)

x_train shape: (8631, 11, 1)
8631 train samples
3699 test samples
y_train shape:  (8631, 2)


In [51]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(20, activation="relu"),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 11, 20)         │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 220)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 220)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           442 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 482 (1.88 KB)

 Trainable params: 482 (1.88 KB)

 Non-trainable params: 0 (0.00 B)

In [53]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(numpy_x_train, numpy_y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6377 - loss: 4.7543 - val_accuracy: 0.8461 - val_loss: 2.4623
Epoch 2/15
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8247 - loss: 2.6818 - val_accuracy: 0.8461 - val_loss: 2.4639
Epoch 3/15
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8282 - loss: 2.6693 - val_accuracy: 0.8472 - val_loss: 2.4611
Epoch 4/15
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8367 - loss: 2.5535 - val_accuracy: 0.8472 - val_loss: 2.4605
Epoch 5/15
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8470 - loss: 2.3855 - val_accuracy: 0.8472 - val_loss: 2.4607
Epoch 6/15
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8422 - loss: 2.4579 - val_accuracy: 0.8472 - val_loss: 2.4572
Epoch 7/15
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8411 - loss: 2.5010 - val_accuracy: 0.8472 - val_loss: 2.4484
Epoch 8/15
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8436 - loss: 2.4538 - val_accuracy: 0.8472 - val_loss:

In [55]:
score = model.evaluate(numpy_x_test, numpy_y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 1.4274117946624756
Test accuracy: 0.8913219571113586
